In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import psycopg2
import snowflake.connector

# PostgreSQL
conn_pg = psycopg2.connect(
    host="localhost",
    database="fleetlogix",
    user="postgres",
    password="Spriest123"
)
cur_pg = conn_pg.cursor()

# Snowflake
conn_sf = snowflake.connector.connect(
    user='ENZOZAMBON',
    password='AdiiraelSpriest12345',
    account='GTUWIRG-PU45327',
    warehouse='FLEETLOGIX_WH',
    database='FleetLogix_dhw',
    schema='ANALYTICS'
)
cur_sf = conn_sf.cursor()

In [4]:
def insert_in_batches(cursor, sql, data, batch_size=5000, table_name=""):
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        cursor.executemany(sql, batch)
        conn_sf.commit()
        print(f"[{datetime.now()}] {table_name} - batch {i//batch_size+1} success - {len(batch)} registros")

In [5]:
df_deliveries = pd.read_sql("SELECT * FROM deliveries", conn_pg)
df_trip = pd.read_sql("SELECT * FROM trips", conn_pg)
df_vehicle = pd.read_sql("SELECT * FROM vehicles", conn_pg)
df_driver = pd.read_sql("SELECT * FROM drivers", conn_pg)
df_route = pd.read_sql("SELECT * FROM routes", conn_pg)
df_maint = pd.read_sql("SELECT vehicle_id, MAX(maintenance_date) AS last_maintenance_date FROM maintenance GROUP BY vehicle_id", conn_pg)

C:\Users\Enzo\AppData\Local\Temp\ipykernel_22124\2084301068.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_deliveries = pd.read_sql("SELECT * FROM deliveries", conn_pg)
C:\Users\Enzo\AppData\Local\Temp\ipykernel_22124\2084301068.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_trip = pd.read_sql("SELECT * FROM trips", conn_pg)
C:\Users\Enzo\AppData\Local\Temp\ipykernel_22124\2084301068.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_vehicle = pd.read_sql("SELECT * FROM vehicles", conn_pg)
C:\Users\Enzo\

In [6]:
df_deliveries['scheduled_datetime'] = pd.to_datetime(df_deliveries['scheduled_datetime'])
df_trip['departure_datetime'] = pd.to_datetime(df_trip['departure_datetime'])
df_trip['arrival_datetime'] = pd.to_datetime(df_trip['arrival_datetime'])

In [13]:
# --------------------------- Dimensión VEHICLES ---------------------------
# Convertir acquisition_date a datetime
df_vehicle['acquisition_date'] = pd.to_datetime(df_vehicle['acquisition_date'], errors='coerce')

# Traer la última fecha de mantenimiento por vehículo
df_maint['last_maintenance_date'] = pd.to_datetime(df_maint['last_maintenance_date'], errors='coerce')
df_maint_last = df_maint.groupby('vehicle_id', as_index=False)['last_maintenance_date'].max()

# Merge seguro con df_vehicle
if 'last_maintenance_date' in df_vehicle.columns:
    df_vehicle = df_vehicle.drop(columns=['last_maintenance_date'])

df_vehicle = df_vehicle.merge(df_maint_last, on='vehicle_id', how='left')

# Calcular edad en meses
df_vehicle['age_months'] = ((pd.Timestamp.today() - df_vehicle['acquisition_date']).dt.days // 30).fillna(0).astype(int)

# Asegurarse de que last_maintenance_date exista
if 'last_maintenance_date' not in df_vehicle.columns:
    df_vehicle['last_maintenance_date'] = pd.NaT

# Convertir fechas a string para Snowflake
df_vehicle['acquisition_date'] = df_vehicle['acquisition_date'].dt.strftime('%Y-%m-%d %H:%M:%S')
df_vehicle['last_maintenance_date'] = df_vehicle['last_maintenance_date'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Preparar registros para insert
vehicle_records = list(df_vehicle[['vehicle_id','license_plate','vehicle_type','capacity_kg','fuel_type',
                                   'acquisition_date','age_months','status','last_maintenance_date']].itertuples(index=False, name=None))

# Insertar en batches
insert_in_batches(cur_sf, """
INSERT INTO dim_vehicle (vehicle_id, license_plate, vehicle_type, capacity_kg, fuel_type, acquisition_date, age_months, status, last_maintenance_date)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
""", vehicle_records, batch_size=5000, table_name="dim_vehicle")


[2025-10-14 21:24:07.227150] dim_vehicle - batch 1 success - 200 registros


In [14]:
# --------------------------- Dimensión DRIVERS ---------------------------

# Convertir fechas a datetime
df_driver['hire_date'] = pd.to_datetime(df_driver['hire_date'], errors='coerce')
df_driver['license_expiry'] = pd.to_datetime(df_driver['license_expiry'], errors='coerce')

# Calcular experiencia en meses
df_driver['experience_months'] = ((pd.Timestamp.today() - df_driver['hire_date']).dt.days // 30).fillna(0).astype(int)

# Crear full_name
df_driver['full_name'] = df_driver['first_name'].fillna('') + ' ' + df_driver['last_name'].fillna('')

# Contar viajes por conductor
df_trip_counts = df_trip.groupby('driver_id').size().reset_index(name='total_trips')

# Hacer merge
if 'total_trips' in df_driver.columns:
    df_driver = df_driver.drop(columns=['total_trips'])

df_driver = df_driver.merge(df_trip_counts, on='driver_id', how='left')

# Rellenar NaN con 0
df_driver['total_trips'] = df_driver['total_trips'].fillna(0)
df_driver['completed_trips'] = df_driver['total_trips']

# Categoría de desempeño evitando división por 0
df_driver['performance_category'] = np.where(
    df_driver['total_trips'] == 0, 'Bajo',  # Si no hizo viajes
    np.where(df_driver['completed_trips']/df_driver['total_trips'] > 0.7, 'Alto',
             np.where(df_driver['completed_trips']/df_driver['total_trips'] > 0.5, 'Medio','Bajo'))
)
    
# Convertir fechas a string para Snowflake (maneja NaT)
df_driver['hire_date'] = df_driver['hire_date'].dt.strftime('%Y-%m-%d %H:%M:%S')
df_driver['license_expiry'] = df_driver['license_expiry'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Reemplazar NaN por None para Snowflake
df_driver = df_driver.where(pd.notnull(df_driver), None)

# Preparar registros
driver_records = list(df_driver[['driver_id', 'employee_code','full_name','license_number','license_expiry',
                                 'phone','hire_date','experience_months','status','performance_category']].itertuples(index=False, name=None))

# Insertar en batches
insert_in_batches(cur_sf, """
INSERT INTO dim_driver (driver_id, employee_code, full_name, license_number, license_expiry, phone, hire_date, experience_months, status, performance_category)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)

""", driver_records, batch_size=5000, table_name="dim_driver")

[2025-10-14 21:35:35.602156] dim_driver - batch 1 success - 400 registros


In [15]:
# --------------------------- Dimensión ROUTES ---------------------------
# Llenar valores nulos en numéricos
df_route[['distance_km','estimated_duration_hours','toll_cost']] = df_route[['distance_km','estimated_duration_hours','toll_cost']].fillna(0)

# Asegurar tipos
df_route['distance_km'] = df_route['distance_km'].astype(float)
df_route['estimated_duration_hours'] = df_route['estimated_duration_hours'].astype(float)
df_route['toll_cost'] = df_route['toll_cost'].astype(float)

# Renombrar columnas si es necesario
df_route = df_route.rename(columns={
    'origin': 'origin_city',
    'destination': 'destination_city'
})

# Preparar registros
route_records = list(df_route[['route_id', 'route_code','origin_city','destination_city','distance_km','estimated_duration_hours','toll_cost']].itertuples(index=False, name=None))

# Insertar en batches
insert_in_batches(cur_sf, """
INSERT INTO dim_route (route_id, route_code, origin_city, destination_city, distance_km, estimated_duration_hours, toll_cost)
VALUES (%s,%s,%s,%s,%s,%s,%s)
""", route_records, batch_size=5000, table_name="dim_route")


[2025-10-14 21:35:37.915123] dim_route - batch 1 success - 50 registros


In [17]:
# --------------------------- Dimensión CUSTOMER ---------------------------
df_trip = df_trip.merge(df_route[['route_id','destination_city']], on='route_id', how='left')
df_customer = df_deliveries.merge(df_trip[['trip_id','destination_city']], on='trip_id', how='left')
df_customer_group = df_customer.groupby(['customer_name','destination_city']).size().reset_index(name='total_deliveries')

def cat_func(x):
    if x==1: return 'Ocasional'
    elif x==2: return 'Regular'
    else: return 'Habitual'

df_customer_group['customer_category'] = df_customer_group['total_deliveries'].apply(cat_func)
customers_dim = [(i+1, row['customer_name'], row['destination_city'], datetime.today(), row['total_deliveries'], row['customer_category']) 
                 for i,row in df_customer_group.iterrows()]

insert_in_batches(cur_sf, """
INSERT INTO dim_customer (customer_key, customer_name, city, first_delivery_date, total_deliveries, customer_category)
VALUES (%s,%s,%s,%s,%s,%s)
""", customers_dim, batch_size=5000, table_name="dim_customer")

[2025-10-14 21:36:06.696098] dim_customer - batch 1 success - 5000 registros
[2025-10-14 21:36:09.168827] dim_customer - batch 2 success - 5000 registros
[2025-10-14 21:36:11.702932] dim_customer - batch 3 success - 5000 registros
[2025-10-14 21:36:14.433855] dim_customer - batch 4 success - 5000 registros
[2025-10-14 21:36:16.742602] dim_customer - batch 5 success - 5000 registros
[2025-10-14 21:36:19.188247] dim_customer - batch 6 success - 5000 registros
[2025-10-14 21:36:21.570374] dim_customer - batch 7 success - 5000 registros
[2025-10-14 21:36:23.946140] dim_customer - batch 8 success - 5000 registros
[2025-10-14 21:36:26.433988] dim_customer - batch 9 success - 5000 registros
[2025-10-14 21:36:28.904192] dim_customer - batch 10 success - 5000 registros
[2025-10-14 21:36:30.955389] dim_customer - batch 11 success - 5000 registros
[2025-10-14 21:36:33.277792] dim_customer - batch 12 success - 5000 registros
[2025-10-14 21:36:35.633953] dim_customer - batch 13 success - 5000 regis

In [19]:
# --------------------------- Dimensión DATE ---------------------------
all_dates = set(df_deliveries['scheduled_datetime'].dt.date)
dim_date = []

for dt in all_dates:
    dim_date.append((
        int(dt.strftime("%Y%m%d")),
        datetime.combine(dt, datetime.min.time()),
        dt.isoweekday(),
        dt.strftime("%A"),
        dt.day,
        dt.timetuple().tm_yday,
        dt.isocalendar()[1],
        dt.month,
        dt.strftime("%B"),
        (dt.month-1)//3+1,
        dt.year,
        dt.weekday()>=5,
        0, None,
        (dt.month-1)//3+1,
        dt.year
    ))

insert_in_batches(cur_sf, """
INSERT INTO dim_date (date_key, full_date, day_of_week, day_name, day_of_month, day_of_year, week_of_year, month_num, month_name, quarter, year, is_weekend, is_holiday, holiday_name, fiscal_quarter, fiscal_year)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
""", dim_date, batch_size=5000, table_name="dim_date")

[2025-10-14 21:37:08.449184] dim_date - batch 1 success - 1018 registros


In [18]:
# --------------------------- Dimensión TIME ---------------------------
dim_time = []

for h in range(24):
    for m in range(60):
        time_key = h*10000 + m*100
        am_pm = 'AM' if h < 12 else 'PM'
        hour_24 = f"{h:02d}:{m:02d}"
        hour_12 = f"{h%12 if h%12!=0 else 12:02d}:{m:02d} {am_pm}"
        tod = 'Madrugada' if 0 <= h < 6 else 'Mañana' if 6 <= h < 12 else 'Tarde' if 12 <= h < 18 else 'Noche'
        is_business = 8 <= h <= 18
        shift = 'Turno 1' if 0 <= h < 8 else 'Turno 2' if 8 <= h < 16 else 'Turno 3'
        dim_time.append((time_key, h, m, 0, tod, hour_24, hour_12, am_pm, is_business, shift))

insert_in_batches(cur_sf, """
INSERT INTO dim_time (time_key, hour, minute, second, time_of_day, hour_24, hour_12, am_pm, is_business_hour, shift)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
""", dim_time, batch_size=5000, table_name="dim_time")

[2025-10-14 21:37:07.132687] dim_time - batch 1 success - 1440 registros


In [20]:
# ==========================
# 📦 FACT DELIVERIES - ETL
# ==========================

import pandas as pd
import numpy as np
import logging
from snowflake.connector.pandas_tools import write_pandas
import psycopg2
import snowflake.connector

from datetime import timedelta


# ----------------------------------------------------
# 🔹 Logging
# ----------------------------------------------------
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ----------------------------------------------------
# 🔹 Parámetros de ETL
# ----------------------------------------------------
etl_run_id = 1
fuel_price = 3.5  # Precio promedio por litro de combustible

# ----------------------------------------------------
# 🔹 Conexión a PostgreSQL (origen)
# ----------------------------------------------------
conn_pg = psycopg2.connect(
    host="localhost",
    database="fleetlogix",
    user="postgres",
    password="Spriest123"
)

# ----------------------------------------------------
# 🔹 Conexión a Snowflake (destino)
# ----------------------------------------------------
conn_sf = snowflake.connector.connect(
    user='ENZOZAMBON',
    password='AdiiraelSpriest12345',
    account='GTUWIRG-PU45327',
    warehouse='FLEETLOGIX_WH',
    database='FleetLogix_dhw',
    schema='ANALYTICS'
)

# ----------------------------------------------------
# 🔹 Extracción de datos históricos
# ----------------------------------------------------
query = """
SELECT 
    d.delivery_id,
    d.trip_id,
    d.tracking_number,
    d.customer_name,
    d.delivery_address,
    d.package_weight_kg,
    d.scheduled_datetime,
    d.delivered_datetime,
    d.delivery_status,
    d.revenue,
    t.vehicle_id,
    t.driver_id,
    t.route_id,
    t.departure_datetime AS start_datetime,
    t.arrival_datetime AS end_datetime,
    t.fuel_consumed_liters AS fuel_consumed_liters,
    t.total_weight_kg,
    t.status AS trip_status,
    r.distance_km,
    r.estimated_duration_hours,
    r.toll_cost
FROM deliveries AS d
JOIN trips AS t ON d.trip_id = t.trip_id
JOIN routes AS r ON t.route_id = r.route_id
"""
df = pd.read_sql(query, conn_pg)

logger.info(f"📥 Datos extraídos: {len(df)} filas")

# ----------------------------------------------------
# 🔹 Preprocesamiento
# ----------------------------------------------------
df.columns = df.columns.str.upper()

datetime_cols = ['SCHEDULED_DATETIME', 'DELIVERED_DATETIME', 'START_DATETIME', 'END_DATETIME']
for col in datetime_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')

float_cols = [
    'PACKAGE_WEIGHT_KG', 'DISTANCE_KM', 'FUEL_CONSUMED_LITERS',
    'ESTIMATED_DURATION_HOURS', 'TOLL_COST', 'REVENUE', 'TOTAL_WEIGHT_KG'
]
df[float_cols] = df[float_cols].astype(float)

df['RECIPIENT_SIGNATURE'] = df['DELIVERY_STATUS'] != 'cancelled'
df['REVENUE'] = np.where(df['DELIVERY_STATUS']=='cancelled', 0.0, df['REVENUE'])

# ----------------------------------------------------
# 🔹 Cargar dimensiones desde Snowflake
# ----------------------------------------------------
df_vehicle = pd.read_sql("SELECT VEHICLE_ID, VEHICLE_KEY FROM DIM_VEHICLE", conn_sf)
df_driver = pd.read_sql("SELECT DRIVER_ID, DRIVER_KEY FROM DIM_DRIVER", conn_sf)
df_route = pd.read_sql("SELECT ROUTE_ID, ROUTE_KEY FROM DIM_ROUTE", conn_sf)
df_customer = pd.read_sql("SELECT CUSTOMER_NAME, CUSTOMER_KEY FROM DIM_CUSTOMER", conn_sf)

for d in [df_vehicle, df_driver, df_route, df_customer]:
    d.columns = d.columns.str.upper()

# ----------------------------------------------------
# 🔹 Transformación
# ----------------------------------------------------
def transform_fact_deliveries(deliveries_df, dim_vehicles, dim_drivers, dim_routes, dim_customers, etl_run_id):
    logger.info("🔄 Transformando FACT_DELIVERIES")
    fact = deliveries_df.copy()

    # 🔸 Claves de tiempo y fecha
    fact['DATE_KEY'] = pd.to_datetime(fact['DELIVERED_DATETIME']).dt.strftime('%Y%m%d').astype(int)
    fact['SCHEDULED_TIME_KEY'] = pd.to_datetime(fact['SCHEDULED_DATETIME']).dt.strftime('%H%M').astype(int)
    fact['DELIVERED_TIME_KEY'] = pd.to_datetime(fact['DELIVERED_DATETIME']).dt.strftime('%H%M').astype(int)

    # 🔸 Duraciones
    fact['DELIVERY_TIME_MINUTES'] = ((fact['DELIVERED_DATETIME'] - fact['SCHEDULED_DATETIME']).dt.total_seconds() / 60).fillna(0).astype(int)
    fact['DELAY_MINUTES'] = fact['DELIVERY_TIME_MINUTES'].apply(lambda x: max(0, x - 30))
    fact['DELIVERIES_PER_HOUR'] = (60 / fact['DELIVERY_TIME_MINUTES'].clip(lower=1)).round(2)

    # 🔸 Variación de consumo de combustible (±25%)
    fact['FUEL_CONSUMED_LITERS'] = fact['FUEL_CONSUMED_LITERS'] * np.random.uniform(0.75, 1.25, len(fact))

    # 🔸 Eficiencia
    base_efficiency = fact['DISTANCE_KM'] / fact['FUEL_CONSUMED_LITERS'].clip(lower=0.5)
    random_factor = np.random.uniform(0.8, 1.2, len(fact))
    fact['FUEL_EFFICIENCY_KM_PER_LITER'] = (base_efficiency * random_factor).round(2)

# 🔸 Métricas financieras ajustadas
    base_cost = (fact['FUEL_CONSUMED_LITERS']*fuel_price + fact['TOLL_COST'] + fact['DISTANCE_KM']*0.25 + 10)
    fact['COST_PER_DELIVERY'] = (base_cost * np.random.uniform(0.75,1.25,len(fact))).round(2)

# Revenue realista: costo + margen 5%-25%
    fact['REVENUE_PER_DELIVERY'] = np.where(
    fact['DELIVERY_STATUS']=='cancelled', 
    0.0,
    (fact['COST_PER_DELIVERY'] * np.random.uniform(1.05,1.25,len(fact))).round(2)
)

# Ganancia
    fact['PROFIT_PER_DELIVERY'] = (fact['REVENUE_PER_DELIVERY'] - fact['COST_PER_DELIVERY']).round(2)

    # 🔸 Indicadores
    fact['IS_ON_TIME'] = fact['DELAY_MINUTES'] == 0
    fact['IS_DAMAGED'] = np.random.choice([True, False], size=len(fact), p=[0.08, 0.92])
    fact['HAS_SIGNATURE'] = fact['RECIPIENT_SIGNATURE']
    fact['DELIVERY_STATUS'] = np.where(fact['IS_ON_TIME'], 'ON_TIME', 'DELAYED')

    # 🔸 Joins con dimensiones
    fact = fact.merge(dim_vehicles[['VEHICLE_ID','VEHICLE_KEY']], on='VEHICLE_ID', how='left')
    fact = fact.merge(dim_drivers[['DRIVER_ID','DRIVER_KEY']], on='DRIVER_ID', how='left')
    fact = fact.merge(dim_routes[['ROUTE_ID','ROUTE_KEY']], on='ROUTE_ID', how='left')
    fact = fact.merge(dim_customers[['CUSTOMER_NAME','CUSTOMER_KEY']], on='CUSTOMER_NAME', how='left')

    # 🔸 Control ETL
    fact['ETL_BATCH_ID'] = etl_run_id
    fact['ETL_TIMESTAMP'] = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')

    # 🔸 Columnas finales
    fact_deliveries = fact[[
        'DATE_KEY','SCHEDULED_TIME_KEY','DELIVERED_TIME_KEY',
        'VEHICLE_KEY','DRIVER_KEY','ROUTE_KEY','CUSTOMER_KEY',
        'DELIVERY_ID','TRIP_ID','TRACKING_NUMBER',
        'PACKAGE_WEIGHT_KG','DISTANCE_KM','FUEL_CONSUMED_LITERS',
        'DELIVERY_TIME_MINUTES','DELAY_MINUTES','DELIVERIES_PER_HOUR',
        'FUEL_EFFICIENCY_KM_PER_LITER','COST_PER_DELIVERY','REVENUE_PER_DELIVERY','PROFIT_PER_DELIVERY',
        'IS_ON_TIME','IS_DAMAGED','HAS_SIGNATURE','DELIVERY_STATUS',
        'ETL_BATCH_ID','ETL_TIMESTAMP'
    ]].copy()

    logger.info(f"📊 FACT_DELIVERIES transformada: {len(fact_deliveries)} registros")
    return fact_deliveries

# ----------------------------------------------------
# 🔹 Transformar y cargar
# ----------------------------------------------------
fact_df = transform_fact_deliveries(df, df_vehicle, df_driver, df_route, df_customer, etl_run_id)

success, nchunks, nrows, _ = write_pandas(conn_sf, fact_df, 'FACT_DELIVERIES')
logger.info(f"✅ Insertadas {nrows} filas en FACT_DELIVERIES")


INFO:snowflake.connector.connection:Snowflake Connector for Python Version: 4.0.0, Python Version: 3.13.0, Platform: Windows-10-10.0.19043-SP0
INFO:snowflake.connector.connection:Connecting to GLOBAL Snowflake domain
C:\Users\Enzo\AppData\Local\Temp\ipykernel_22124\2529720248.py:79: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn_pg)
INFO:__main__:📥 Datos extraídos: 125850 filas
C:\Users\Enzo\AppData\Local\Temp\ipykernel_22124\2529720248.py:104: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_vehicle = pd.read_sql("SELECT VEHICLE_ID, VEHICLE_KEY FROM DIM_VEHICLE", conn_sf)
C:\Users\Enzo\AppData\Local\Temp\ipykernel_22124\2529720248.py:105: UserWarnin

In [ ]:
import pandas as pd
import psycopg2
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
from datetime import datetime
import schedule
import time
import json

# ===============================================
# CONFIGURACIÓN
# ===============================================
POSTGRES_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'fleetlogix',
    'user': 'postgres',
    'password': 'Spriest123'
}

SNOWFLAKE_CONFIG = {
    "user": 'ENZOZAMBON',
    "password": 'AdiiraelSpriest12345',
    "account": 'GTUWIRG-PU45327',
    "warehouse": 'FLEETLOGIX_WH',
    "database": 'FleetLogix_dhw',
    "schema": 'ANALYTICS'
}

LOAD_HOUR = "10:24"
BATCH_SIZE = 5000

# ===============================================
# CONEXIONES
# ===============================================
def connect_postgres():
    return psycopg2.connect(**POSTGRES_CONFIG)

def connect_snowflake():
    return snowflake.connector.connect(**SNOWFLAKE_CONFIG)

# ===============================================
# FUNCIONES
# ===============================================
def insert_staging(conn_sf, df):
    """Inserta registros en staging_daily_load como VARIANT JSON"""
    if df is None or df.empty:
        return
    cur = conn_sf.cursor()
    for record in df.to_dict(orient='records'):
        raw_json = json.dumps(record, default=str)
        cur.execute(
            "INSERT INTO staging_daily_load (raw_data, load_timestamp) VALUES (PARSE_JSON(%s), CURRENT_TIMESTAMP)",
            (raw_json,)
        )
    cur.close()

def get_unique_from_staging(conn_sf, id_field):
    """
    Obtiene registros únicos de staging por id_field usando ROW_NUMBER
    Se accede al campo de JSON correctamente con raw_data:id_field
    """
    cur = conn_sf.cursor()
    query = f"""
        SELECT raw_data
        FROM (
            SELECT raw_data,
                   ROW_NUMBER() OVER (PARTITION BY raw_data:{id_field} ORDER BY load_timestamp DESC) AS rn
            FROM staging_daily_load
        )
        WHERE rn = 1
    """
    cur.execute(query)
    rows = cur.fetchall()
    cur.close()

    records = []
    for r in rows:
        v = r[0]
        if isinstance(v, dict):
            records.append(v)
        else:
            try:
                records.append(json.loads(v))
            except:
                continue
    return pd.DataFrame(records)

def merge_dimension_sf(conn_sf, df, table, key_column):
    """Upsert simple a Snowflake dimension"""
    if df is None or df.empty:
        return
    cur = conn_sf.cursor()
    columns = df.columns.tolist()
    for i in range(0, len(df), BATCH_SIZE):
        batch = df.iloc[i:i+BATCH_SIZE]
        merge_sql = f"""
            MERGE INTO {table} t
            USING (SELECT {', '.join(['%s AS '+col for col in columns])}) s
            ON t.{key_column} = s.{key_column}
            WHEN MATCHED THEN
                UPDATE SET {', '.join([f"{col}=s.{col}" for col in columns if col != key_column])}
            WHEN NOT MATCHED THEN
                INSERT ({', '.join(columns)}) VALUES ({', '.join(['s.'+col for col in columns])})
        """
        cur.executemany(merge_sql, batch.values.tolist())
    cur.close()
    print(f"✅ Dimensión {table} actualizada ({len(df)} filas)")

# ===============================================
# TRANSFORMACIONES DIMENSIONALES
# ===============================================
def transform_dim_vehicles(df_vehicle, df_maint):
    if df_vehicle is None or df_vehicle.empty:
        return pd.DataFrame()
    df_vehicle['acquisition_date'] = pd.to_datetime(df_vehicle['acquisition_date'], errors='coerce')
    if df_maint is None or df_maint.empty:
        df_maint = pd.DataFrame(columns=['vehicle_id','maintenance_date'])
    df_maint['maintenance_date'] = pd.to_datetime(df_maint['maintenance_date'], errors='coerce')
    last_maint = df_maint.groupby('vehicle_id', as_index=False)['maintenance_date'].max()
    df_vehicle = df_vehicle.drop(columns=['last_maintenance_date'], errors='ignore')
    if not last_maint.empty:
        last_maint = last_maint.rename(columns={'maintenance_date':'last_maintenance_date'})
        df_vehicle = df_vehicle.merge(last_maint, on='vehicle_id', how='left')
    else:
        df_vehicle['last_maintenance_date'] = pd.NaT
    df_vehicle['age_months'] = ((pd.Timestamp.today() - df_vehicle['acquisition_date']).dt.days // 30).fillna(0).astype(int)
    df_vehicle['acquisition_date'] = df_vehicle['acquisition_date'].dt.strftime('%Y-%m-%d %H:%M:%S')
    df_vehicle['last_maintenance_date'] = pd.to_datetime(df_vehicle['last_maintenance_date'], errors='coerce') \
                                           .dt.strftime('%Y-%m-%d %H:%M:%S')
    df_vehicle = df_vehicle.where(pd.notnull(df_vehicle), None)
    return df_vehicle

# ===============================================
# ETL DIARIO CON STAGING EN SNOWFLAKE
# ===============================================
def run_daily_etl():
    conn_pg = connect_postgres()
    conn_sf = connect_snowflake()
    
    # 🔹 Extraer desde PostgreSQL (incremental usando departure_datetime)
    fecha_inicio = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    df_vehicle = pd.read_sql("SELECT * FROM vehicles", conn_pg)
    df_driver = pd.read_sql("SELECT * FROM drivers", conn_pg)
    df_route = pd.read_sql("SELECT * FROM routes", conn_pg)
    
    # Cambio clave: trips usa departure_datetime
    df_trip = pd.read_sql(
        "SELECT * FROM trips WHERE departure_datetime > %s",
        conn_pg,
        params=[fecha_inicio]
    )
    
    df_deliveries = pd.read_sql("SELECT * FROM deliveries", conn_pg)
    df_maint = pd.read_sql("SELECT * FROM maintenance", conn_pg)
    
    # Insertar en staging
    for df in [df_vehicle, df_driver, df_route, df_trip, df_deliveries, df_maint]:
        insert_staging(conn_sf, df)
    
    # Obtener registros únicos
    df_vehicle = get_unique_from_staging(conn_sf, 'vehicle_id')
    df_driver = get_unique_from_staging(conn_sf, 'driver_id')
    df_route = get_unique_from_staging(conn_sf, 'route_id')
    df_trip = get_unique_from_staging(conn_sf, 'trip_id')
    df_deliveries = get_unique_from_staging(conn_sf, 'delivery_id')
    df_maint = get_unique_from_staging(conn_sf, 'vehicle_id')
    
    # Transformar dimensiones
    df_vehicle_t = transform_dim_vehicles(df_vehicle, df_maint)
    
    # Merge en Snowflake
    merge_dimension_sf(conn_sf, df_vehicle_t, 'dim_vehicle', 'vehicle_id')
    
    # Transformar fact table y cargar
    df_fact = df_deliveries.copy()
    if not df_fact.empty:
        df_fact['ETL_TIMESTAMP'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        write_pandas(conn_sf, df_fact, 'FACT_DELIVERIES')
        print(f"✅ {len(df_fact)} filas insertadas en FACT_DELIVERIES")
    
    conn_pg.close()
    conn_sf.close()
    print(f"✅ ETL finalizado - {datetime.now()}")

# ===============================================
# SCHEDULER
# ===============================================
schedule.every().day.at(LOAD_HOUR).do(run_daily_etl)
print(f"⏰ Scheduler activo, ETL se ejecutará cada día a las {LOAD_HOUR}")

while True:
    schedule.run_pending()
    time.sleep(60)





⏰ Scheduler activo, ETL se ejecutará cada día a las 10:24


In [ ]:
#------ EXTRACREDIT ------

### El script sirve para predecir el consumo de combustible de un viaje, usando datos históricos de entregas, vehículos y rutas. 
# Esto ayuda a FleetLogix a planificar mejor los viajes, optimizar costos de combustible y mejorar la eficiencia operativa.

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import snowflake.connector

# ===============================
# 1️⃣ Conexión a Snowflake
# ===============================
conn = snowflake.connector.connect(
    user='ENZOZAMBON',
    password='AdiiraelSpriest12345',
    account='GTUWIRG-PU45327',
    warehouse='FLEETLOGIX_WH',
    database='FleetLogix_dhw',
    schema='ANALYTICS'
)

# ===============================
# 2️⃣ Consulta de datos relevantes
# ===============================
query = """
SELECT 
    F.DISTANCE_KM,
    F.DELIVERY_TIME_MINUTES,
    V.VEHICLE_TYPE,
    V.CAPACITY_KG,
    V.AGE_MONTHS,
    R.ESTIMATED_DURATION_HOURS,
    F.FUEL_CONSUMED_LITERS
FROM FACT_DELIVERIES F
JOIN DIM_VEHICLE V ON F.VEHICLE_KEY = V.VEHICLE_KEY
JOIN DIM_ROUTE R ON F.ROUTE_KEY = R.ROUTE_KEY
WHERE F.FUEL_CONSUMED_LITERS IS NOT NULL
"""
df = pd.read_sql(query, conn)

# Convertir columnas a minúsculas para trabajar más cómodo en pandas
df.columns = df.columns.str.lower()

# ===============================
# 3️⃣ Preparación de datos
# ===============================
X = df.drop(columns=['fuel_consumed_liters'])
y = df['fuel_consumed_liters']

categorical_features = ['vehicle_type']
numerical_features = ['distance_km', 'delivery_time_minutes', 'capacity_kg', 'age_months', 'estimated_duration_hours']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

# ===============================
# 4️⃣ División de datos en train/test
# ===============================
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ===============================
# 5️⃣ Pipeline + Modelo
# ===============================
model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42))
])

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# ===============================
# 6️⃣ Evaluación
# ===============================
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Absolute Error: {mae:.2f} litros")
print(f"R² Score: {r2:.2f}")

# ===============================
# 7️⃣ Predicción de un nuevo viaje
# ===============================
nuevo_viaje = pd.DataFrame({
    'distance_km': [120],
    'delivery_time_minutes': [150],
    'vehicle_type': ['Camión mediano'],
    'capacity_kg': [3000],
    'age_months': [36],
    'estimated_duration_hours': [2.5]
})

consumo_estimado = model.predict(nuevo_viaje)
print(f"Consumo estimado de combustible: {consumo_estimado[0]:.2f} litros")



C:\Users\Enzo\AppData\Local\Temp\ipykernel_12708\2578783964.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Mean Absolute Error: 19.48 litros
R² Score: 0.91
Consumo estimado de combustible: 19.36 litros
